In [6]:
import requests

r = requests.get('https://raw.githubusercontent.com/shaunhegarty/word-ladder/master/wordladder/resources/ladder-common-3.json')


In [7]:
import json
data = json.loads(r.content.decode('utf-8'))


In [9]:
list(data.keys())[:100]

['car-now',
 'big-van',
 'bar-low',
 'per-top',
 'gun-joe',
 'run-she',
 'and-six',
 'new-say',
 'and-win',
 'sir-bob',
 'get-war',
 'day-cup',
 'run-old',
 'hot-run',
 'how-ice',
 'son-pay',
 'war-due',
 'via-her',
 'now-ten',
 'set-due',
 'due-now',
 'sun-lee',
 'cup-get',
 'era-put',
 'six-van',
 'end-use',
 'top-had',
 'bar-via',
 'pop-ice',
 'sir-may',
 'did-who',
 'won-and',
 'won-are',
 'say-top',
 'not-day',
 'led-hot',
 'key-lot',
 'man-cup',
 'men-say',
 'lee-for',
 'boy-per',
 'own-new',
 'ran-see',
 'ice-men',
 'too-san',
 'may-few',
 'way-one',
 'art-fox',
 'day-his',
 'bob-air',
 'oil-saw',
 'bob-out',
 'out-man',
 'did-war',
 'way-joe',
 'gas-off',
 'was-now',
 'act-led',
 'era-saw',
 'use-for',
 'man-his',
 'tax-red',
 'see-any',
 'may-act',
 'men-now',
 'own-bob',
 'him-say',
 'who-ten',
 'the-cut',
 'see-lot',
 'hot-oil',
 'cut-day',
 'she-fox',
 'ran-due',
 'due-for',
 'not-was',
 'god-sir',
 'run-air',
 'his-off',
 'ice-him',
 'own-may',
 'day-use',
 'led-boy',
 'jo

In [1]:
import sys
sys.path.append('/workspace/')

from sqlalchemy.orm import Session
from sqlalchemy.dialects.postgresql import insert
from anagrammer import models
from anagrammer.database import engine

models.Base.metadata.create_all(bind=engine)





In [4]:
with Session(engine) as session:
    with open('/workspace/dictionaries/common.frequency.csv') as dictionary:
        source = "wikipedia-word-frequency-list-2019"
        values = []
        for line in dictionary:
            word, frequency = line.split()
            word = word.strip()
            values.append({'word': word, 
                            'frequency': int(frequency),
                            'word_length': len(word),
                            'dictionary': 'common',
                            'source': source,
                            })
        statement = insert(models.Dictionary).values(values).on_conflict_do_nothing()
        session.execute(statement)
        session.commit()
    
    with open('/workspace/dictionaries/sowpods.txt') as dictionary:
        values = []
        for word in dictionary:
            word = word.strip()
            values.append({'word': word, 
                            'word_length': len(word),
                            'dictionary': 'sowpods',
                            })
        statement = insert(models.Dictionary).values(values).on_conflict_do_nothing()
        session.execute(statement)
        session.commit()

In [21]:
with Session(engine) as session:
    results = session.query(models.Dictionary).filter(models.Dictionary.frequency > 50000).all()
    print(results)

[<anagrammer.models.Dictionary object at 0x7f00ef383940>, <anagrammer.models.Dictionary object at 0x7f00ef3838b0>, <anagrammer.models.Dictionary object at 0x7f00ef3839d0>, <anagrammer.models.Dictionary object at 0x7f00ef383a30>, <anagrammer.models.Dictionary object at 0x7f00ef383a90>, <anagrammer.models.Dictionary object at 0x7f00ef383ac0>, <anagrammer.models.Dictionary object at 0x7f00ef383b50>, <anagrammer.models.Dictionary object at 0x7f00ef383be0>, <anagrammer.models.Dictionary object at 0x7f00ef383c70>, <anagrammer.models.Dictionary object at 0x7f00ef383d00>, <anagrammer.models.Dictionary object at 0x7f00ef383d90>, <anagrammer.models.Dictionary object at 0x7f00ef383e20>, <anagrammer.models.Dictionary object at 0x7f00ef383eb0>, <anagrammer.models.Dictionary object at 0x7f00ef383f40>, <anagrammer.models.Dictionary object at 0x7f00f0d98040>, <anagrammer.models.Dictionary object at 0x7f00f0d980a0>, <anagrammer.models.Dictionary object at 0x7f00f0d98130>, <anagrammer.models.Dictionary 